In [ ]:
import heapq

# Define the grid size
rows = 4
cols = 4
num_nodes = rows * cols

# Define the grid weights (example weights)
grid_weights = [
    [1, 3, 2, 4],
    [2, 1, 4, 3],
    [3, 2, 1, 2],
    [4, 3, 2, 1]
]

# Define the neighbors for each node
neighbors = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def dijkstra_shortest_path(origin, destination, weights, neighbors):
    distances = [float('inf')] * num_nodes
    distances[origin] = 0

    priority_queue = [(0, origin)]  # (distance, node)
    
    while priority_queue:
        dist, node = heapq.heappop(priority_queue)
        
        if node == destination:
            break
        
        for dr, dc in neighbors:
            r, c = divmod(node, cols)
            nr, nc = r + dr, c + dc
            
            if 0 <= nr < rows and 0 <= nc < cols:
                neighbor_node = nr * cols + nc
                new_dist = dist + weights[nr][nc]
                
                if new_dist < distances[neighbor_node]:
                    distances[neighbor_node] = new_dist
                    heapq.heappush(priority_queue, (new_dist, neighbor_node))

    # Reconstruct the path
    path = [destination]
    node = destination
    while node != origin:
        for dr, dc in neighbors:
            r, c = divmod(node, cols)
            nr, nc = r + dr, c + dc
            
            if 0 <= nr < rows and 0 <= nc < cols:
                neighbor_node = nr * cols + nc
                if distances[node] == distances[neighbor_node] + weights[r][c]:
                    path.append(neighbor_node)
                    node = neighbor_node
                    break

    path.reverse()
    return path

origin = 0
destination = num_nodes - 1
shortest_path = dijkstra_shortest_path(origin, destination, grid_weights, neighbors)
print("Shortest path:", shortest_path)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

rows = 4
cols = 4
node_positions = np.array([(r, c) for r in range(rows) for c in range(cols)])

plt.figure(figsize=(6, 6))
plt.scatter(node_positions[:, 1], node_positions[:, 0], color='b', marker='o', s=300)

for r in range(rows):
    for c in range(cols):
        plt.text(c, r, f'{r * cols + c + 1}', ha='center', va='center', color='white', fontsize=12)

plt.xticks(np.arange(cols))
plt.yticks(np.arange(rows))
plt.grid()
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# %pip install arcgis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

# Set the font to Times New Roman
rc('font', family='Times New Roman')
# Data
categories = ['Drive alone', 'Carpool', 'Transit', 'Walk & Bike']
trip_distance_has_disability = [5.6, 6.39, 8.46, 0.94]
trip_distance_no_disability = [6.76, 5.76, 11.52, 1.14]
trip_frequency_has_disability = [5.76, 5.88, 4.21, 5.81]
trip_frequency_no_disability = [6, 7.2, 5.71, 6.7]

bar_width = 0.3  # Width of the bars
index = np.arange(len(categories))  # X-axis values

# Create a single figure and divide it into subplots
fig, (ax_distance, ax_frequency) = plt.subplots(1, 2, figsize=(9, 5), sharex=True)

# Plot trip distance data
bars1 = ax_distance.bar(index - bar_width/2, trip_distance_has_disability, bar_width, label='Has Disability')
bars2 = ax_distance.bar(index + bar_width/2, trip_distance_no_disability, bar_width, label="Doesn't have Disability")
ax_distance.set_ylabel('Trip Distance (Miles)')
ax_distance.set_title('Avg. Trip Distance')
ax_distance.set_xticks(index)
ax_distance.set_xticklabels(categories)
ax_distance.legend()
# Plot trip frequency data
bars3=ax_frequency.bar(index - bar_width/2, trip_frequency_has_disability, bar_width, label='Has Disability')
bars4=ax_frequency.bar(index + bar_width/2, trip_frequency_no_disability, bar_width, label="Doesn't have Disability")
ax_frequency.set_xlabel('Travel Modes')
ax_frequency.set_ylabel('Trip Frequency per day (#)')
ax_frequency.set_title('Avg. Trip Frequency per day')
ax_frequency.set_xticks(index)
ax_frequency.set_xticklabels(categories)

# Add bar values on top of the bars
for bar in bars1:
    height = bar.get_height()
    ax_distance.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')
for bar in bars2:
    height = bar.get_height()
    ax_distance.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')
for bar in bars3:
    height = bar.get_height()
    ax_frequency.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')
for bar in bars4:
    height = bar.get_height()
    ax_frequency.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')

# Adjust layout
plt.tight_layout()
# Save the figure in high quality
plt.savefig('C:/Users/graduate/Box/Paper/Mode choice behaviour for people with disability in Utah/Data HHTS2012 and codes/results/trip dist and trip freq by modes.png', dpi=300)

# Show the plot
plt.show()


In [ ]:
import requests
import folium

# Read Bing Maps API Key from the text file
with open('C:/Users/graduate/Box/Paper to study for 10 sept/Travel demand modelling/Papers/mode choice/TRB2024/HHTS2012/codes_biogeme/API.txt', 'r') as key_file:
    BING_MAPS_API_KEY = key_file.read().strip()

# Coordinates for origin and destination
origin = (37.7749, -122.4194)  # Change to your origin coordinates
destination = (34.0522, -118.2437)  # Change to your destination coordinates

# Request route information from Bing Maps API
route_url = f"https://dev.virtualearth.net/REST/v1/Routes/Driving?wp.0={origin[0]},{origin[1]}&wp.1={destination[0]},{destination[1]}&key={BING_MAPS_API_KEY}"
response = requests.get(route_url)
data = response.json()

# Extract route points from the response
route_legs = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['itineraryItems']
route_points = [(item['maneuverPoint']['coordinates'][0], item['maneuverPoint']['coordinates'][1]) for item in route_legs]

# Create a folium map centered around the origin
m = folium.Map(location=origin, zoom_start=10)

# Add polyline for the route
folium.PolyLine(locations=route_points, color='blue').add_to(m)

# Add markers for origin and destination
folium.Marker(location=origin, popup='Origin', icon=folium.Icon(color='blue')).add_to(m)
folium.Marker(location=destination, popup='Destination', icon=folium.Icon(color='red')).add_to(m)

# Display the map
m.save('map_with_points.html')


In [ ]:
import requests
import folium
import pandas as pd

# Read Bing Maps API Key from the text file
with open('C:/Users/graduate/Box/Paper to study for 10 sept/Travel demand modelling/Papers/mode choice/TRB2024/HHTS2012/codes_biogeme/API.txt', 'r') as key_file:
    BING_MAPS_API_KEY = key_file.read().strip()

# Read waypoint coordinates from the CSV file
data = pd.read_csv('data.csv')
waypoints = [tuple(map(float, coord.split(','))) for coord in data['coordinates']]

# Make a copy of the first waypoint to complete the loop
waypoints.append(waypoints[0])

# Build the waypoints string for the API request
waypoints_str = '&'.join([f'waypoint.{i}={coord[0]},{coord[1]}' for i, coord in enumerate(waypoints)])

# Request route information from Bing Maps API
route_url = f"https://dev.virtualearth.net/REST/v1/Routes/Driving?{waypoints_str}&key={BING_MAPS_API_KEY}"
response = requests.get(route_url)
data = response.json()

# Extract route points from the response
route_legs = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['itineraryItems']
route_points = [(item['maneuverPoint']['coordinates'][0], item['maneuverPoint']['coordinates'][1]) for item in route_legs]

# Create a folium map centered around the first waypoint
m = folium.Map(location=waypoints[0], zoom_start=10)

# Add polyline for the route
folium.PolyLine(locations=route_points, color='blue').add_to(m)

# Add markers for waypoints
for i, waypoint in enumerate(waypoints):
    popup_text = f'Point {i + 1}'
    folium.Marker(location=waypoint, popup=popup_text, icon=folium.Icon(color='green')).add_to(m)

# Display the map
m.save('completed_route_map.html')

## R to python

In [1]:
import pandas as pd

# Read CSV files
trip = pd.read_csv("Utah2012survey_trip.csv")
hh = pd.read_csv("Utah2012survey_hh.csv")
person = pd.read_csv("Utah2012survey_person.csv")

# Display first few rows
print(trip.head())
print(hh.head())
print(person.head())

# Display column names
print(trip.columns)
print(hh.columns)
print(person.columns)

# Filter data using pandas
trip2 = trip[trip['limitmobil'] == 1]
print(trip2.shape)

trip2 = trip2.iloc[:, 0:191]  # Select columns 1 to 191
print(trip2.columns.to_list())

# Perform transformations
trip2['o_fixed'] = 0
trip2.loc[trip2['o_purp_t'].isin(["drop-pickup", "home", "school-daycare", "work", "work-related"]), 'o_fixed'] = 1

trip2['d_fixed'] = 0
trip2.loc[trip2['d_purp_t'].isin(["drop-pickup", "home", "school-daycare", "work", "work-related"]), 'd_fixed'] = 1
trip2.head()

# Filter and manipulate data
trip_both_fixed = trip2[(trip2['o_fixed'] == 1) & (trip2['d_fixed'] == 1)]
print(trip_both_fixed.head())
print(trip_both_fixed.shape)

trip_both_fixed = trip_both_fixed[trip_both_fixed['h_CO_NAME_v30'].isin(["DAVIS", "UTAH", "WEBER", "SALT LAKE"])]
print(trip_both_fixed.shape)

trip_either_fixed = trip2[((trip2['o_fixed'] == 1) & (trip2['d_fixed'] == 0)) | ((trip2['o_fixed'] == 0) & (trip2['d_fixed'] == 1))]
print(trip_either_fixed['o_fixed'].value_counts())
print(trip_either_fixed['d_fixed'].value_counts())
print(trip_either_fixed.shape)
trip_either_fixed = trip_either_fixed[trip_either_fixed['h_CO_NAME_v30'].isin(["DAVIS", "UTAH", "WEBER", "SALT LAKE"])]
print(trip_either_fixed.shape)

# Write to CSV
trip_both_fixed.to_csv("trip_both_fixed.csv", index=False)
trip_either_fixed.to_csv("trip_either_fixed.csv", index=False)

# Display column names and selected rows
print(trip_either_fixed.columns.tolist())

print(trip_either_fixed.head())
print(trip_either_fixed.iloc[:, list(range(4)) + list(range(191, 193)) + [125, 158]].head(10))


# Load the selected columns from trip_either_fixed
trip_either_select = trip_either_fixed[['hptripid', 'tripID', 'hhmemberid', 'o_purp_t', 'depart_hhm', 'mode_t', 'hhsize', 'workers', 'hh_income', 'num_veh', 'age', 'gender', 'emp_text',
                                         'education', 'licensed', 'limitmobil', 'h_CoTAZID_v30', 'o_CoTAZID_v30', 'o_fixed', 'd_fixed', 'd_purp_t', 'arrive_hhm', 'd_CoTAZID_v30']]
print(trip_either_select.head())
print(trip_either_select[trip_either_select['hptripid'] == "W17604ZB.02.03"])

# Create a new dataframe to store merged rows
merged_data = pd.DataFrame(columns=trip_either_select.columns)

# Iterate through unique hhmemberid values
unique_ids = trip_either_select['hhmemberid'].unique()
for hhmemberid in unique_ids:
    subset_data = trip_either_select[trip_either_select['hhmemberid'] == hhmemberid]

    if subset_data.empty or 'o_fixed' not in subset_data.columns or 'd_fixed' not in subset_data.columns:
        continue
    
    i = 0
    while i < len(subset_data):
        if pd.notnull(subset_data.iloc[i]['o_fixed']) and subset_data.iloc[i]['o_fixed'] == 1:
            j = i + 1
            while j < len(subset_data):
                if pd.notnull(subset_data.iloc[j]['d_fixed']) and subset_data.iloc[j]['d_fixed'] == 1:
                    new_row = subset_data.iloc[i][['hptripid', 'tripID', 'hhmemberid', 'o_purp_t', 'depart_hhm', 'mode_t', 'hhsize', 'workers', 'hh_income', 'num_veh', 'age', 'gender', 'emp_text',
                                                    'education', 'licensed', 'limitmobil', 'h_CoTAZID_v30', 'o_CoTAZID_v30', 'o_fixed']].append(subset_data.iloc[j][['d_fixed', 'd_purp_t', 'arrive_hhm', 'd_CoTAZID_v30']])
                    
                    if new_row['hptripid'] not in merged_data['hptripid'].values:
                        merged_data = merged_data.append(new_row, ignore_index=True)
                    
                    i = j + 1
                    break
                j += 1
            
            if j >= len(subset_data):
                i += 1
        else:
            i += 1
# Display merged data
print(merged_data.head())

# Adjust arrive_hhm values
trip_new = merged_data.copy()
trip_new['arrive_hhm'] = trip_new.apply(lambda row: row['arrive_hhm'] + 2400 if row['depart_hhm'] > row['arrive_hhm'] else row['arrive_hhm'], axis=1)

# Sort by hptripid
trip_new = trip_new.sort_values(by='hptripid')

# Write to CSV
trip_new.to_csv("trip_either_fixed_new.csv", index=False)

# Select columns from trip_both_fixed
trip_both_fixed_select = trip_both_fixed[['hptripid', 'tripID', 'hhmemberid', 'o_purp_t', 'depart_hhm', 'mode_t', 'hhsize', 'workers', 'hh_income', 'num_veh', 'age', 'gender', 'emp_text',
                                           'education', 'licensed', 'limitmobil', 'h_CoTAZID_v30', 'o_CoTAZID_v30', 'o_fixed', 'd_fixed', 'd_purp_t', 'arrive_hhm', 'd_CoTAZID_v30']]

# Concatenate dataframes
trip_new = pd.concat([trip_both_fixed_select, merged_data], ignore_index=True)

# Sort by hptripid
trip_new = trip_new.sort_values(by='hptripid')

# Write to CSV
trip_new.to_csv("trip_both_and_either_new.csv", index=False)


# Read CSV files
trip_new = pd.read_csv("trip_both_and_either_new.csv")
print(trip_new.head())

TAZ = pd.read_csv("TAZ_point.csv")
TAZ = TAZ[['CO_TAZID', 'ACRES', 'DEVACRES', 'DEVPBLEPCT', 'X', 'Y', 'CO_NAME', 'CITY_NAME']]
print(TAZ.head())

TAZ_o = TAZ.copy()
TAZ_d = TAZ.copy()

TAZ_o2 = trip_new.merge(TAZ_o, left_on='o_CoTAZID_v30', right_on='CO_TAZID', how='left')
print(TAZ_o2.head())
print(TAZ_o2.shape)

TAZ_d2 = trip_new.merge(TAZ_d, left_on='d_CoTAZID_v30', right_on='CO_TAZID', how='left')
print(TAZ_d2.head())
print(TAZ_d2.shape)

TAZ_o2.to_csv("TAZ_origin.csv", index=False)
TAZ_d2.to_csv("TAZ_destination.csv", index=False)

# Minimum activity duration
trip2_flexible = trip2[trip2['d_fixed'] != 1]
print(trip2.shape)
print(trip2_flexible.shape)
print(trip2_flexible.head())

C:\Users\graduate\AppData\Local\Temp\ipykernel_13316\887388986.py:4: DtypeWarning: Columns (13,46) have mixed types. Specify dtype option on import or set low_memory=False.
  trip = pd.read_csv("Utah2012survey_trip.csv")


   record_id        hptripid  tripID   hhmemberid  password     weight  \
0          1  C10021WH.01.01       1  C10021WH.01  C10021WH  52.867523   
1          2  C10021WH.01.02       2  C10021WH.01  C10021WH  52.867523   
2          3  C10021WH.01.03       3  C10021WH.01  C10021WH  52.867523   
3          4  C10021WH.01.04       4  C10021WH.01  C10021WH  52.867523   
4          5  C10021WH.01.05       5  C10021WH.01  C10021WH  52.867523   

   recl_hbw  resptype  diarytype  PURP_PER  ...  a_SUBAREA_v20 a_CO_FIPS_v20  \
0         0         1          1         1  ...              2             5   
1         0         1          1        17  ...              2             5   
2         0         1          1        17  ...              2             5   
3         0         1          1        18  ...              2             5   
4         0         1          1        18  ...              2             5   

  a_CO_NAME_v20 a_CITY_NAME_v20 a_DISTSUP_v20  a_DSUP_NAME_v20 a_DISTLRG_v

C:\Users\graduate\AppData\Local\Temp\ipykernel_13316\887388986.py:82: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_row = subset_data.iloc[i][['hptripid', 'tripID', 'hhmemberid', 'o_purp_t', 'depart_hhm', 'mode_t', 'hhsize', 'workers', 'hh_income', 'num_veh', 'age', 'gender', 'emp_text',
C:\Users\graduate\AppData\Local\Temp\ipykernel_13316\887388986.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(new_row, ignore_index=True)
C:\Users\graduate\AppData\Local\Temp\ipykernel_13316\887388986.py:82: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_row = subset_data.iloc[i][['hptripid', 'tripID', 'hhmemberid', 'o_purp_t', 'depart_hhm', 'mode_t', 'hhsize', 'workers', 'hh_income', 'num_veh

         hptripid tripID   hhmemberid     o_purp_t depart_hhm mode_t hhsize  \
0  M10155BD.02.01      1  M10155BD.02         home       1020   auto      2   
1  M11010EU.01.01      1  M11010EU.01         home       1145   auto      1   
2  M11010EU.01.04      4  M11010EU.01  drop-pickup       1835   auto      1   
3  M12288ED.01.01      1  M12288ED.01         home       1100   auto      4   
4  M13229DW.04.01      1  M13229DW.04         home        830   auto      6   

  workers hh_income num_veh  ... education licensed limitmobil h_CoTAZID_v30  \
0       1         5       2  ...         3        1          1        490880   
1       0         5       1  ...         6        1          1        490438   
2       0         5       1  ...         6        1          1        490438   
3       2        -1       3  ...         3        1          1        490531   
4       1        -1       4  ...         1        2          1        490881   

  o_CoTAZID_v30 o_fixed d_fixed     d_purp_t